<a href="https://colab.research.google.com/github/Ann-ap/sirius/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Установка необходимых библиотек
!pip install transformers torch sentencepiece rouge_score nltk streamlit PyPDF2

In [ ]:

%%writefile app.py
import streamlit as st
from PyPDF2 import PdfReader
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch

Overwriting app.py


In [ ]:

MAX_FILE_SIZE = 5 * 1024 * 1024  # 5 MB

SUPPORTED_LANGUAGES = {
    "Русский": "ru_RU",
    "Английский": "en_XX",
    "Французский": "fr_XX",
    "Немецкий": "de_DE",
    "Испанский": "es_XX"
}

def extract_text_from_pdf(pdf_file):
    try:
        reader = PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ''
        return text
    except Exception as e:
        st.error(f"Error extracting text from PDF: {str(e)}")
        return None

@st.cache(allow_output_mutation=True)
def load_model():
    tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
    model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    return tokenizer, model, device

def summarize_text(text, tokenizer, model, device, summary_type, src_lang, tgt_lang):
    try:
      # Токенизация предложений
      sentence_list = nltk.sent_tokenize(text)

      # Удаление специальных символов и цифр
      formatted_text = re.sub('[^a-zA-Z]', ' ', text)
      formatted_text = re.sub(r'\s+', ' ', formatted_text)

      # Получение стоп-слов
      stopwords = nltk.corpus.stopwords.words('english')

      # Подсчет частоты слов
      word_frequencies = {}
      for word in nltk.word_tokenize(formatted_text):
          if word.lower() not in stopwords:
              if word not in word_frequencies.keys():
                  word_frequencies[word] = 1
              else:
                  word_frequencies[word] += 1

      # Нормализация частоты слов
      maximum_frequency = max(word_frequencies.values())
      for word in word_frequencies.keys():
          word_frequencies[word] = (word_frequencies[word] / maximum_frequency)

      # Подсчет оценок предложений
      sentence_scores = {}
      for sent in sentence_list:
          for word in nltk.word_tokenize(sent.lower()):
              if word in word_frequencies.keys():
                  if len(sent.split(' ')) < 30:
                      if sent not in sentence_scores.keys():
                          sentence_scores[sent] = word_frequencies[word]
                      else:
                          sentence_scores[sent] += word_frequencies[word]

      # Выбор наиболее важных предложений
      summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

      # Объединение предложений в итоговое краткое содержание
      summary = ' '.join(summary_sentences)

      return summary
    except Exception as e:
        st.error(f'Произошла ошибка при суммаризации текста: {str(e)}')
        return None

st.title("Многоязычный суммаризатор текста")

tokenizer, model, device = load_model()

input_option = st.radio(
    "Выберите способ ввода текста",
    ("Загрузить PDF", "Ввести текст")
)

if input_option == "Загрузить PDF":
    uploaded_file = st.file_uploader("Выберите PDF файл (макс. 5 MB)", type="pdf")
    if uploaded_file is not None:
        if uploaded_file.size > MAX_FILE_SIZE:
            st.error(f"Файл слишком большой. Максимальный размер: {MAX_FILE_SIZE/1024/1024} MB")
        else:
            st.info("PDF файл успешно загружен. Извлечение текста...")
            text = extract_text_from_pdf(uploaded_file)
    else:
        text = None
else:
    text = st.text_area("Введите текст для суммаризации", height=200)

if text:
    src_lang = st.selectbox(
        "Выберите язык исходного текста",
        list(SUPPORTED_LANGUAGES.keys()),
        index=0
    )

    tgt_lang = st.selectbox(
        "Выберите язык для суммаризации",
        list(SUPPORTED_LANGUAGES.keys()),
        index=0
    )

    summary_type = st.selectbox(
        "Выберите тип суммаризации",
        ("standard", "brief", "big"),
        format_func=lambda x: {"standard": "Стандартный", "brief": "Краткий", "big": "Расширенный"}[x]
    )

    if st.button("Суммаризировать"):
        st.info("Суммаризация текста...")
        summary = summarize_text(
            text,
            tokenizer,
            model,
            device,
            summary_type,
            SUPPORTED_LANGUAGES[src_lang],
            SUPPORTED_LANGUAGES[tgt_lang]
        )
        if summary:
            st.subheader("Краткое содержание")
            st.text_area("Текст краткого содержания", summary, height=200)

            summary_bytes = summary.encode('utf-8')
            st.download_button(label="Скачать краткое содержание",
                data=summary_bytes,
                file_name=f"summary_{summary_type}_{tgt_lang}.txt",
                mime="text/plain"
            )
        else:
            st.error("Не удалось сгенерировать краткое содержание. Пожалуйста, проверьте введенный текст и попробуйте снова.")
elif input_option == "Загрузить PDF":
    st.error("Не удалось извлечь текст из PDF. Пожалуйста, проверьте, является ли файл корректным, и попробуйте снова.")



2024-11-03 18:46:00.353 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-03 18:46:00.356 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-03 18:46:00.358 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new
caching commands, `st.cache_data` or `st.cache_resource`. More information
[in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching
logic used by `st.cache_data` and `st.cache_resource`. This might lead to some problems
or unexpected behavior in certain edge cases.

2024-11-03 18:46:00.364 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-03 18:46:00.367 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-0

Этот код создает веб-приложение с помощью Streamlit для загрузки PDF-файла, извлечения из него текста и создания краткого содержания.

In [ ]:
ngrok.set_auth_token("2oKmfjGjLjAjUT0jj2vL1RpQJiC_2ETkEsv1JpNrUiDUjeuvq")

In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
import requests

try:
    response = requests.get("http://localhost:8501")
    print(f"Streamlit работает. Статус код: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("Не удалось подключиться к Streamlit")

Streamlit работает. Статус код: 200


In [ ]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"http-http://localhost:8501-7a9b559c-c668-41e2-a566-f6b778e893a3","ID":"398c02a85a8756e0f6446f554f3b9d5b","uri":"/api/tunnels/http-http:%2F%2Flocalhost:8501-7a9b559c-c668-41e2-a566-f6b778e893a3","public_url":"https://ca7c-34-125-140-62.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:8501","inspect":true},"metrics":{"conns":{"count":6,"gauge":1,"rate1":0.0017713631235299395,"rate5":0.008523014632337036,"rate15":0.00500455017929987,"p50":3956564669.5,"p90":97613253768,"p95":97613253768,"p99":97613253768},"http":{"count":10,"rate1":0.0013171314474606022,"rate5":0.01265774787680523,"rate15":0.00804598188385317,"p50":2543802,"p90":351489008.70000005,"p95":376362580,"p99":376362580}}}],"uri":"/api/tunnels"}


In [ ]:
tunnels = ngrok.get_tunnels()
print(f"Активные туннели: {tunnels}")

Активные туннели: [<NgrokTunnel: "https://ca7c-34-125-140-62.ngrok-free.app" -> "http://localhost:8501">]


In [ ]:
ngrok.kill()

In [ ]:
public_url = ngrok.connect(addr='http://localhost:8501')
print(public_url)

NgrokTunnel: "https://65d8-34-125-140-62.ngrok-free.app" -> "http://localhost:8501"
